### catalogo de datos madrid

In [1]:
#parse dcat xml
import xml.etree.ElementTree as ET
import urllib.request

ns = {
    'foaf': "http://xmlns.com/foaf/0.1/",
    'skos': "http://datos.gob.es/kos/sector-publico/sector/",
    'dcat': "http://www.w3.org/ns/dcat#",
    'rdfs': "http://www.w3.org/2000/01/rdf-schema#",
    'rdf': "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    'dct': "http://purl.org/dc/terms/",
    'dc': "http://purl.org/dc/elements/1.1/",
    'time': "http://www.w3.org/2006/time#",
    'xsd': "http://www.w3.org/2001/XMLSchema#",
    'j.0': "http://www.w3.org/1999/xhtml/vocab#",
    'j.1': "http://a9.com/-/spec/opensearch/1.1/",
    'api': "http://purl.org/linked-data/api/vocab#",
}





In [2]:
with urllib.request.urlopen('https://datos.madrid.es/egob/catalogo.rdf') as f:
    tree = ET.parse(f)
    root = tree.getroot()

root.tag

'{http://www.w3.org/1999/02/22-rdf-syntax-ns#}RDF'

In [3]:
from datetime import datetime
ds_list = [{'indentifier': dataset.find('dct:identifier', ns).text,
            'dataset_title': dataset.find('dct:title', ns).text
            , 'theme': dataset.find('dcat:theme', ns).attrib.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
            , 'keywords': dataset.find('dcat:keyword', ns).text
            , 'issued':  datetime.strptime(dataset.find('dct:issued', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'modified': datetime.strptime(dataset.find('dct:modified', ns).text, '%Y-%m-%dT%H:%M:%S')
            , 'frequency': dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns).text if  dataset.find('dct:accrualPeriodicity/dct:Frequency/rdfs:label', ns) is not None else None
            , 'distribution_title': distribution.find('dct:title', ns).text if distribution.find('dct:title', ns) is not None else None
            , 'mediaType': distribution.find('dcat:mediaType', ns).text
            , 'accessURL': distribution.find('dcat:accessURL', ns).text
            }
           for dataset
           in root.findall('./dcat:Catalog/dcat:dataset/dcat:Dataset', ns)
           for distribution
           in dataset.findall('dcat:distribution/dcat:Distribution', ns)

           ]

len(ds_list)

8906

In [4]:
import polars as pl
from datetime import datetime

dfp = pl.DataFrame(ds_list)

#dt = datetime.today().strftime('%Y%m%d')
#dfp.write_csv(f'../data/catalogo_{dt}.csv', separator=';')

### consultas con duckdb

In [5]:


import duckdb
conn = duckdb.connect()


%load_ext sql
%sql conn --alias duckdb
%config SqlMagic.displaylimit = 100


In [6]:
%%sql

copy(
select indentifier,	dataset_title,	theme,	keywords,	issued,	modified,	frequency, distribution_title, mediaType, accessURL, current_date() as fx_data
from dfp
group by all
order by modified desc
--limit 20
) to '../data/catalogo_distribuciones_.csv' (HEADER, DELIMITER ';');

Running query in 'duckdb'

Count
8906


In [7]:
%%sql

copy(
select indentifier,	dataset_title,	theme,	keywords,	issued,	modified,	frequency, count(*) n_dist
from dfp
group by all
order by modified desc
--limit 20
) to '../data/catalogo_datasets_.csv' (HEADER, DELIMITER ';');


Running query in 'duckdb'

Count
616
